In [1]:
from baselines.common.vec_env import VecFrameStack
from comp300.LearningModel.AgentClasses import *
from baselines.common.cmd_util import make_vec_env
import gym
from gym.wrappers.frame_stack import FrameStack
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

from os import listdir
from os.path import isfile, join
import re

/home/patrick/PycharmProjects/comp300/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/patrick/PycharmProjects/comp300/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/patrick/PycharmProjects/comp300/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/patrick/PycharmP

In [2]:
#setup the env
model_path = "/home/patrick/models/BreakoutNoFrameskip-v4-groundTruth"
env_id = 'BreakoutNoFrameskip-v4'
env_type = 'atari'

env = make_vec_env(env_id, env_type, 1, 0,
                   wrapper_kwargs={
                       'clip_rewards': False,
                       'episode_life': False,
                   })
env = VecFrameStack(env, 4)

#load the reward network
trainedNetwork = RewardNetwork("", env=env, env_type=env_type)
trainedNetwork.load_state_dict(torch.load("/home/patrick/models/fullGuiTest/fullTest.params"))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
agent = PPO2Agent(env, 'atari', True)
trainedNetwork.to(device)

#run the agent in the env once and save the ground truth reward and observations
def GetDemoFromAgent(agent, network, env):
    trueReward = 0
    learnedReward = 0

    currentReward = 0
    currentObservation = env.reset()
    timeSteps = 0
    done = False

    #run the demo
    while True:
        trueReward += currentReward
        shapedObservation = torch.from_numpy(currentObservation).float().to(device)
        reward, abs_reward = network.predict_reward(shapedObservation)
        learnedReward += reward.tolist()

        action = agent.act(currentObservation,  currentReward, done)
        currentObservation, currentReward, done, info = env.step(action)
        shapedObservations = currentObservation
        timeSteps += 1

        if done:
            trueReward += currentReward
            reward, abs_reward = network.predict_reward(shapedObservation)
            learnedReward += reward.tolist()
            break
    print("{}, {}".format(trueReward, learnedReward))
    return trueReward, learnedReward

#a method to find all the models in a given dir that are just numbers
def Find_all_Models(model_dir):

    checkpoints = []
    filesandDirs = listdir(model_dir)
    allFiles = []
    for i in filesandDirs:
        if isfile(join(model_dir, i)):
            allFiles.append(i)

    for file in allFiles:
        if re.match('^[0-9]+$',file.title()):
            checkpoints.append(file.title())

    return checkpoints

Logging to /tmp/openai-2020-04-14-19-40-17-615168





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
trueRewards = []
learnedRewards = []

models = Find_all_Models(model_path)

for model in models:
    agent.load(model_path + "/" + model)
    trueReward, learnedReward = GetDemoFromAgent(agent, trainedNetwork, env)
    tf.keras.backend.clear_session()
    trueRewards.append(trueReward[0])
    learnedRewards.append(learnedReward)

[74.], 5807.086614608765
[99.], 8801.28234577179
[302.], 9686.289559841156
[279.], 12355.746445178986
[315.], 10104.987102985382
[311.], 11991.43548154831
[121.], 9723.304317951202
[353.], 12904.598839759827
[262.], 10444.717190265656
[35.], 5633.167418003082
[249.], 8418.554112315178
[324.], 9844.46140730381
[391.], 10147.289002656937
[198.], 8974.429542779922
[78.], 8364.966038703918
[272.], 12268.841401815414
[326.], 12195.57649922371
[338.], 10841.334688067436
[16.], 4279.92800617218
[346.], 7174.848008517176
[278.], 9919.384356021881
[87.], 7814.639692306519
[63.], 7264.450046539307
[356.], 13221.490128993988
[339.], 10248.476876974106
[268.], 9209.965847492218
[378.], 8443.879289865494
[38.], 6081.103522777557
[308.], 9138.051247358322
[284.], 8621.057996749878
[313.], 9617.301353931427
[299.], 7828.881627559662
[321.], 10725.391894340515
[302.], 10865.210080623627
[348.], 10141.492789268494
[159.], 9988.903798103333
[298.], 9585.462687969208
[268.], 10379.92119550705
[279.], 805

In [5]:
import random

correct = 0

for i in range(100):
    ti_index = 0
    tj_index = 0
    while ti_index == tj_index:
        ti_index = random.randint(0, len(learnedRewards)-1)
        tj_index = random.randint(0, len(learnedRewards)-1)
        
    groundOrder = trueRewards[ti_index] <= trueRewards[tj_index]
    learnedOrder = learnedRewards[ti_index] <= learnedRewards[tj_index]
    
    if groundOrder == learnedOrder:
        correct = correct +1
    
print(correct)

71
